In [1]:
# Auto-detect GPUs, set env, and launch the HPO script from Jupyter

import os, sys, subprocess, shlex, json, pathlib

PROJECT_DIR = "/work/projects/myproj/SOLO_Supervised_RFDETR/"            # change if needed
SCRIPT      = "TrainRFDETR_SOLO_SingleClass_STATIC_Ucloud.py"     # the file you pasted earlier

# Which classes to train in this HPO run:
# "leu"  -> only Leucocyte
# "epi"  -> only Squamous Epithelial Cell
# "all"  -> both

HPO_TARGET = "all"   # DETERMINES WHAT TO RUN IN THE HPO!
os.environ.setdefault("RFDETR_HPO_TARGET", HPO_TARGET)
print("RFDETR_HPO_TARGET =", os.environ["RFDETR_HPO_TARGET"])
# Input mode:
#   - "640": full-image mode (no patching)
#   - any other positive integer (e.g. "224"): patch mode with that patch size
RFDETR_INPUT_MODE = os.environ.get("RFDETR_INPUT_MODE", "640")
os.environ["RFDETR_INPUT_MODE"] = RFDETR_INPUT_MODE
mode_key = RFDETR_INPUT_MODE.strip().lower()
# Clear legacy size knobs to avoid stale notebook env affecting runs.
os.environ.pop("RFDETR_PATCH_SIZE", None)
os.environ.pop("RFDETR_FULL_RESOLUTION", None)
os.environ["RFDETR_USE_PATCH_224"] = "0" if mode_key in {"640", "640x640", "full", "full640", "full_640"} else "1"
print("RFDETR_INPUT_MODE =", os.environ["RFDETR_INPUT_MODE"])
print("RFDETR_USE_PATCH_224 =", os.environ["RFDETR_USE_PATCH_224"])

# ---- GPU detection (respect CUDA_VISIBLE_DEVICES if already set) ----
def detect_gpu_count():
    # If the user already masked GPUs, respect that
    vis = os.environ.get("CUDA_VISIBLE_DEVICES", "").strip()
    if vis:
        return len([x for x in vis.split(",") if x.strip() != ""])
    # Try torch
    try:
        import torch
        return int(torch.cuda.device_count())
    except Exception:
        pass
    # Fallback: nvidia-smi
    try:
        out = subprocess.check_output(["nvidia-smi", "-L"], stderr=subprocess.DEVNULL, text=True)
        return len([ln for ln in out.splitlines() if "GPU " in ln])
    except Exception:
        return 0

gpu_count = detect_gpu_count()
if gpu_count == 0:
    print("⚠️ No GPUs detected. The script will run CPU-only or fail if CUDA is required.")
else:
    print(f"✅ Detected {gpu_count} GPU(s)")

# Let the script decide mapping; just hint max parallel = number of visible GPUs
os.environ.setdefault("MAX_PARALLEL", str(max(1, gpu_count)))
# allocator safety
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
# Optional perf toggles (script also sets them per-worker)
os.environ.setdefault("TF32", "1")

# ---- Optional helpful env (edit if your paths differ) ----
# These are only used if your .py doesn’t already find them via defaults.
# os.environ.setdefault("STAT_DATASETS_ROOT", f"{PROJECT_DIR}/SOLO_Supervised_RFDETR/Stat_Dataset")
# os.environ.setdefault("IMAGES_FALLBACK_ROOT", "/work/Member Files:yourname/CellScanData/Zoom10x - Quality Assessment_Cleaned")
# os.environ.setdefault("OUTPUT_ROOT", "/work/Member Files:yourname/RFDETR_SOLO_OUTPUT/HPO_BOTH_OVR")
# os.environ.setdefault("NUM_WORKERS", "8")  # dataloader workers

# ---- Run the script ----
wd = pathlib.Path(PROJECT_DIR).resolve()
py = sys.executable
cmd = f'{shlex.quote(py)} -u {shlex.quote(str(wd / SCRIPT))}'

print("\n[LAUNCH]")
print(" cwd:", wd)
print(" cmd:", cmd)
print(" env: MAX_PARALLEL=", os.environ.get("MAX_PARALLEL"))
print(" env: RFDETR_INPUT_MODE=", os.environ.get("RFDETR_INPUT_MODE"))
print(" env: RFDETR_USE_PATCH_224=", os.environ.get("RFDETR_USE_PATCH_224"))

#proc = subprocess.Popen(cmd, cwd=str(wd), shell=True)
# If you prefer to block and see output inline, use:
subprocess.run(cmd, cwd=str(wd), shell=True, check=True)




✅ Detected 1 GPU(s)

[LAUNCH]
 cwd: C:\work\projects\myproj\SOLO_Supervised_RFDETR
 cmd: 'C:\Users\SH37YE\AppData\Local\anaconda3\envs\AI_PowMic\python.exe' -u 'C:\work\projects\myproj\SOLO_Supervised_RFDETR\TrainRFDETR_SOLO_SingleClass_STATIC_Ucloud.py'
 env: MAX_PARALLEL= 1


NotADirectoryError: [WinError 267] The directory name is invalid